In [8]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

#each table will store an aggregate that will later be combined
means = pd.DataFrame()
counts = pd.DataFrame()
stds = pd.DataFrame()

#looks at every year that ESPN has data for the top recruits
for year in range(2006,2023):
    
    url = 'http://www.espn.com/college-sports/football/recruiting/playerrankings/_/class/'+str(year)+'/order/true'

    soup = bs(requests.get(url).text, 'html.parser')
    td_tags = soup.find_all('td')
    
    #looks at the 4th col for hometown then 8th for grade
    states = [tag.text.split(', ')[1][:2] for tag in td_tags[12::9]]
    grades = [int(tag.text) for tag in td_tags[16::9]]
    df = pd.DataFrame({'State':states, 'Grades':grades})
    
    #saves the aggregates per year
    means[year] = df.groupby(['State']).mean()['Grades']
    counts[year] = df.groupby(['State']).count()['Grades']
    stds[year] = df.groupby(['State']).std()['Grades']

#aggregates the aggreagates
m = means.mean(axis=1).rename('Avg Player Grade')
c = counts.sum(axis=1).rename('# of Recruits')
s = stds.mean(axis=1).rename('Std Dev of Grade')

#saves these aggreagates as a summary per team which can then be filtered by grade, count, or consistency
rankings = pd.concat([m, c, s], axis=1).sort_values(by='Avg Player Grade', ascending=False)
rankings_filt = rankings[(rankings['# of Recruits']>100)&(rankings['Std Dev of Grade']<2.5)]
rankings_filt

,Avg Player Grade,# of Recruits,Std Dev of Grade
State,,,
LA,83.104559,166.0,2.483347
GA,83.063495,410.0,2.407230
TX,83.006995,572.0,2.425304
FL,82.969330,693.0,2.418347
CA,82.900545,407.0,2.409387
NC,82.771804,140.0,1.873897
OH,82.517134,129.0,1.897563
